Data Mining - MSDS 7331 - Thurs 6:30, Summer 2016
Team 3 (AKA Team Super Awesome):  Sal Melendez, Rahn Lieberman, Thomas Rogers


Github page:
https://github.com/RahnL/DataScience-SMU/tree/master/DataMining

<hr>

# Data Mining - Mini Lab 1


- Import the data

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.read_csv("data/LLCP2014XPT.txt", sep="\t", encoding = "ISO-8859-1")
df.head()

## Identifying the Variables of Interest
We are most interested to know how the variables in our dataset relate to self-reported health quality.

We'll work to reduce the dataset and create an imputed variable from the self-reported measure of health. The question of interest is, "Would You Say in General That Your Health is: (1) excellent, (2) very good, (3) good, (4) fair, (5) poor." Choices 7 and 9 were "unsure" and "not asked", respectively.


In [ ]:
df_reduced = df[['_AGE80','GENHLTH','_INCOMG', '_SMOKER3']]

# this creates a new variable by categorizing GENHLTH into three categories
#df_reduced['health'] = pd.cut(df_reduced.GENHLTH,[0,2,5,9],3,labels=[0,1,2]) 

df_reduced.head()
df_reduced.info()
df_reduced.describe().transpose()

df_reduced.dropna(inplace=True) # this drops those empty variables

# Genhlth, _incomg, _smoker3 are all categorical and integer already, so one-hot-encoding is not needed.

## Training And Testing

Threefold cross validation, using 80% of data as training.

In [ ]:
from sklearn.cross_validation import ShuffleSplit

#... setup x, y
#if '_Age80' in df_reduced:
y = df_reduced['_AGE80'].values # get the labels we want
del df_reduced['_AGE80'] # get rid of the class label
X = df_reduced.values # use everything else to predict!

# do the cross validation
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n=num_instances,
                         n_iter=num_cv_iterations,
                         test_size  = 0.2)
                         
print (cv_object)

ShuffleSplit(464660, n_iter=3, test_size=0.2, random_state=None)


## Regression and stuff...

Do a logistic regression on our data.


In [ ]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object: 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print ("====Iteration",iter_num," ====")
    print ("accuracy", acc) 
    print ("confusion matrix\n",conf)
    iter_num+=1

## SVM
Go do a support vector machine.
